# USDA Data & Coordinates Data 

In [1]:
import pandas as pd

In [ ]:
df_usda = pd.read_csv('../data/raw/usda_corn_yield_iowa_2016_2023.csv')
df_coords = pd.read_csv('../data/raw/county_coordinates.csv')

# Shape check for both 
print(f"USDA data shape : {df_usda.shape}")
print(f"Coordinates data : {df_coords.shape}")

USDA data shape : (824, 8)
Coordinates data : (3220, 3)


In [66]:
df_usda.head()

,year,state_alpha,county_name,county_code,commodity_desc,statisticcat_desc,Value,unit_desc
0,2019,IA,OTHER (COMBINED) COUNTIES,998,CORN,YIELD,191.2,BU / ACRE
1,2023,IA,BUENA VISTA,21,CORN,YIELD,212.0,BU / ACRE
2,2022,IA,BUENA VISTA,21,CORN,YIELD,196.9,BU / ACRE
3,2021,IA,BUENA VISTA,21,CORN,YIELD,203.9,BU / ACRE
4,2020,IA,BUENA VISTA,21,CORN,YIELD,183.9,BU / ACRE


In [67]:
df_coords.head()

,county_code,latitude,longitude
0,1001,32.532237,-86.646440
1,1003,30.659218,-87.746067
2,1005,31.870253,-85.405104
3,1007,33.015893,-87.127148
4,1009,33.977358,-86.566440


In [68]:
df_usda['county_code'].unique()

array([998,  21,  35,  41,  59,  63, 119, 141, 143, 147, 149, 151, 167,
        23,  33,  67,  69,  81,  91, 109, 131, 189, 195, 197,   5,  13,
        17,  19,  37,  43,  55,  61,  65,  89, 191,   9,  25,  27,  47,
        73,  77,  85,  93, 133, 161, 165, 193,  15,  49,  75,  79,  83,
        99, 127, 153, 157, 169, 171, 187,  11,  31,  45,  95,  97, 103,
       105, 113, 139, 163,   1,   3,  29,  71, 129, 137, 145, 155, 173,
         7,  39,  53, 117, 121, 125, 135, 159, 175, 181, 185,  51,  57,
        87, 101, 107, 111, 115, 123, 177, 179, 183])

For this data we will be getting the corrdinates for the counties that are not classified as "OTHER COUNTIES" for the county_name column and we know these are the counties with county_code that are less than 998. 

In [69]:
df_usda = df_usda[df_usda['county_code'] < 998].copy()

In [70]:
# Check the shape after filtereing
df_usda.shape

(811, 8)

We will use a static standard information that will be mapping all of the states to the initial part of the code that is seen in the coordinates data but not present in our corn yield data found from USDA.

In [71]:
# This is static, standard information.
state_fips_map = {
    'AL': '01', 'AK': '02', 'AZ': '04', 'AR': '05', 'CA': '06', 'CO': '08', 'CT': '09', 'DE': '10', 'FL': '12',
    'GA': '13', 'HI': '15', 'ID': '16', 'IL': '17', 'IN': '18', 'IA': '19', 'KS': '20', 'KY': '21', 'LA': '22',
    'ME': '23', 'MD': '24', 'MA': '25', 'MI': '26', 'MN': '27', 'MS': '28', 'MO': '29', 'MT': '30', 'NE': '31',
    'NV': '32', 'NH': '33', 'NJ': '34', 'NM': '35', 'NY': '36', 'NC': '37', 'ND': '38', 'OH': '39', 'OK': '40',
    'OR': '41', 'PA': '42', 'RI': '44', 'SC': '45', 'SD': '46', 'TN': '47', 'TX': '48', 'UT': '49', 'VT': '50',
    'VA': '51', 'WA': '53', 'WV': '54', 'WI': '55', 'WY': '56'
}

In [72]:
df_usda['state_fips_str'] = df_usda['state_alpha'].map(state_fips_map)

In [73]:
df_usda['county_fips_str'] = df_usda['county_code'].astype(str).str.zfill(3)

In [74]:
df_usda['full_fips_code'] = df_usda['state_fips_str'] + df_usda['county_fips_str']

In [75]:
df_usda['full_fips_code'] = df_usda['full_fips_code'].astype(int)

In [76]:
df_coords.head()

,county_code,latitude,longitude
0,1001,32.532237,-86.646440
1,1003,30.659218,-87.746067
2,1005,31.870253,-85.405104
3,1007,33.015893,-87.127148
4,1009,33.977358,-86.566440


In [77]:
df_usda.head()

,year,state_alpha,county_name,county_code,commodity_desc,statisticcat_desc,Value,unit_desc,state_fips_str,county_fips_str,full_fips_code
1,2023,IA,BUENA VISTA,21,CORN,YIELD,212.0,BU / ACRE,19,021,19021
2,2022,IA,BUENA VISTA,21,CORN,YIELD,196.9,BU / ACRE,19,021,19021
3,2021,IA,BUENA VISTA,21,CORN,YIELD,203.9,BU / ACRE,19,021,19021
4,2020,IA,BUENA VISTA,21,CORN,YIELD,183.9,BU / ACRE,19,021,19021
5,2019,IA,BUENA VISTA,21,CORN,YIELD,190.8,BU / ACRE,19,021,19021


In [78]:
df_merged = pd.merge(
    left = df_usda, 
    right = df_coords, 
    left_on = 'full_fips_code', 
    right_on = 'county_code',
    how = "inner"
)

In [79]:
df_merged.head()

,year,state_alpha,county_name,county_code_x,commodity_desc,statisticcat_desc,Value,unit_desc,state_fips_str,county_fips_str,full_fips_code,county_code_y,latitude,longitude
0,2023,IA,BUENA VISTA,21,CORN,YIELD,212.0,BU / ACRE,19,021,19021,19021,42.741522,-95.141432
1,2022,IA,BUENA VISTA,21,CORN,YIELD,196.9,BU / ACRE,19,021,19021,19021,42.741522,-95.141432
2,2021,IA,BUENA VISTA,21,CORN,YIELD,203.9,BU / ACRE,19,021,19021,19021,42.741522,-95.141432
3,2020,IA,BUENA VISTA,21,CORN,YIELD,183.9,BU / ACRE,19,021,19021,19021,42.741522,-95.141432
4,2019,IA,BUENA VISTA,21,CORN,YIELD,190.8,BU / ACRE,19,021,19021,19021,42.741522,-95.141432


In [80]:
df_merged.drop(columns = ['county_code_y'], inplace = True)
df_merged.rename(columns = {'county_code_x': 'county_code'}, inplace = True)

In [81]:
df_merged.shape

(811, 13)

In [83]:
df_merged[['year', 'county_name', 'Value', 'latitude', 'longitude']]

,year,county_name,Value,latitude,longitude
0,2023,BUENA VISTA,212.0,42.741522,-95.141432
1,2022,BUENA VISTA,196.9,42.741522,-95.141432
2,2021,BUENA VISTA,203.9,42.741522,-95.141432
3,2020,BUENA VISTA,183.9,42.741522,-95.141432
4,2019,BUENA VISTA,190.8,42.741522,-95.141432
...,...,...,...,...,...
806,2020,WASHINGTON,185.4,41.329401,-91.725052
807,2019,WASHINGTON,178.9,41.329401,-91.725052
808,2018,WASHINGTON,214.3,41.329401,-91.725052
809,2017,WASHINGTON,220.4,41.329401,-91.725052


In [ ]:
ouput_path = '../data/processed/iowa_yield_with_coords.csv'
df_merged.to_csv(ouput_path, index = False)